# 🧪 Chatbot de Biologia — RAG + Guardrails + Fallback + UI

Módulo completo em **4 aulas** para construir um chatbot de **Biologia** com:
- RAG simples (Sentence-Transformers + FAISS)
- Geração com LLM primária (T5)
- **Validação** com Pydantic + heurísticas (mini-guardrails)
- **Fallback Router** para um segundo modelo (FLAN-T5)
- **Interface** com Gradio

Cada aula possui objetivo, conceitos e código pronto para rodar no **Google Colab** ou **Jupyter**.


## ✅ Visão geral
1) **RAG**: embeddings (Sentence-Transformers) + índice FAISS para recuperar trechos relevantes.
2) **LLM primária**: T5 (`t5-small`) gerando respostas em português usando **apenas o contexto** recuperado.
3) **Validação**: Pydantic + heurísticas (tamanho mínimo, frases proibidas) + aderência semântica resposta↔contexto.
4) **Fallback router**: se reprovar na validação, tenta **FLAN-T5** com prompt alternativo.
5) **UI**: Gradio para interação (mostra modo *primary/fallback*, confiança e citações).


---
## 📘 Aula 1 — RAG de Biologia (embeddings + FAISS)

**Objetivo**: Montar a base de conhecimento de Biologia, criar embeddings e indexar no FAISS; implementar `retrieve()`.

**Conceitos & técnicas**:
- Embedding semântico (Sentence-Transformers)
- Índice vetorial (FAISS)
- Top-k retrieval (busca dos trechos mais relevantes)
- Cosine via dot product: normalizar vetores + `IndexFlatIP`


In [2]:
# Célula 1 — Instalação de dependências essenciais para RAG
!pip install -q sentence-transformers faiss-cpu numpy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 17.7 MB/s eta 0:00:00


In [3]:
# Célula 2 — Imports, base de biologia e seed
from sentence_transformers import SentenceTransformer
import faiss, numpy as np, random, os

random.seed(42); np.random.seed(42)

# Base de documentos (curtos) — BIOLOGIA
documents = [
    "Fotossíntese é o processo em que plantas, algas e cianobactérias convertem energia luminosa em química, produzindo glicose e oxigênio a partir de CO2 e água.",
    "DNA é uma molécula em dupla hélice que armazena informação genética; genes são segmentos que codificam proteínas.",
    "Mitose é a divisão celular que gera duas células-filhas idênticas: prófase, metáfase, anáfase e telófase.",
    "Biodiversidade é a variedade de espécies, genes e ecossistemas; alta diversidade tende a aumentar a resiliência ecológica.",
    "Cadeias alimentares descrevem o fluxo de energia: produtores, consumidores e decompositores.",
    "O bioma Amazônia tem alta pluviosidade, floresta densa e enorme diversidade de espécies.",
    "Sucessão ecológica é a mudança gradual na comunidade ao longo do tempo, podendo ser primária ou secundária.",
    "Enzimas são catalisadores biológicos que aceleram reações químicas e apresentam especificidade por substrato.",
    "A membrana plasmática é uma bicamada lipídica com proteínas; controla difusão, osmose e transporte ativo.",
    "Seleção natural aumenta a frequência de características vantajosas em populações ao longo das gerações.",
    "Taxonomia organiza organismos: domínio, reino, filo, classe, ordem, família, gênero e espécie.",
    "Ciclos biogeoquímicos (carbono, nitrogênio) circulam elementos essenciais entre biosfera, atmosfera, hidrosfera e litosfera."
]
len(documents)


12

In [ ]:
# Célula 3 — Embeddings normalizados + índice FAISS (produto interno ≈ cosseno)
embedder = SentenceTransformer('all-MiniLM-L6-v2')

doc_emb = embedder.encode(documents, convert_to_numpy=True, normalize_embeddings=True)
d = doc_emb.shape[1]

index = faiss.IndexFlatIP(d)         # Produto interno (com vetores normalizados = cosseno)
index.add(doc_emb)                    # Indexa os documentos
print("Docs indexados:", index.ntotal)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Docs indexados: 12


In [ ]:
# Célula 4 — Função de recuperação (top-k) com scores (similaridade)
def retrieve(query: str, k: int = 3):
    q_emb = embedder.encode([query], convert_to_numpy=True, normalize_embeddings=True)
    sims, idx = index.search(q_emb, k)     # sims ~ cosine similarity (0..1)
    hits = [(int(i), float(s)) for i, s in zip(idx[0], sims[0])]
    return hits

# Teste rápido
query = "Explique o processo de fotossíntese e seus produtos."
hits = retrieve(query, k=3)
for rank, (i, s) in enumerate(hits, 1):
    print(f"{rank:02d}  score={s:.3f}  doc={documents[i]}")


01  score=0.707  doc=Fotossíntese é o processo em que plantas, algas e cianobactérias convertem energia luminosa em química, produzindo glicose e oxigênio a partir de CO2 e água.
02  score=0.562  doc=Cadeias alimentares descrevem o fluxo de energia: produtores, consumidores e decompositores.
03  score=0.454  doc=Seleção natural aumenta a frequência de características vantajosas em populações ao longo das gerações.


---
## 🤖 Aula 2 — LLM Primária (T5) + Geração usando apenas o contexto

**Objetivo**: Gerar uma resposta em PT-BR usando **somente** os trechos recuperados (RAG → Geração).

**Conceitos & técnicas**:
- Prompt restritivo ("use apenas o contexto; se faltar, diga 'Não encontrei no contexto.'")
- Truncation e beam search
- Citações: devolver os índices dos docs usados


In [ ]:
# Célula 5 — Instalação de dependências para geração
!pip install -q transformers torch


In [ ]:
# Célula 6 — T5 primário (leve) + geração
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM # AutoTokenizer → Converte texto em números (tokens) , AutoModelForSeq2SeqLM → Carrega modelo T5 para geração de texto

PRIMARY_MODEL_ID = "t5-small"  # leve e rápido para aula , geração de texto em português com base no contexto recuperado do RAG.
tok_primary = AutoTokenizer.from_pretrained(PRIMARY_MODEL_ID) # Baixa e carrega o tokenizer do T5-small
mdl_primary = AutoModelForSeq2SeqLM.from_pretrained(PRIMARY_MODEL_ID) # Baixa e carrega o modelo T5-small na memória

def build_context(hits):
    # Junta os top-k em um bloco de contexto curto , Carrega os Docs normalizados da RAG , juntando com espaços entre eles (Vetores)
    ctx_parts = [documents[i] for i, _ in hits]
    return " ".join(ctx_parts)

def generate_primary(question: str, hits, max_in=512, max_out=160): #512 é o número máximo que esse modelo suporta de tokens com contexto
    context = build_context(hits)
    prompt = (
        "Responda em português de forma objetiva usando APENAS o contexto. " # Importância do Prompt , definir saida.
        "Se a informação não estiver no contexto, diga: 'Não encontrei no contexto.' " #Fallback caso não encontre resposta
        f"Pergunta: {question} " #Prompt inputado
        f"Contexto: {context}" # Formata os documentos recuperados da RAG
    )
    inputs = tok_primary(prompt, return_tensors='pt', truncation=True, max_length=max_in) # Transforma Texto em Tokens
    outputs = mdl_primary.generate(**inputs, max_length=max_out, num_beams=5, early_stopping=True) # Usa o modelo Primario T5 para gerar a resposta
    answer = tok_primary.decode(outputs[0], skip_special_tokens=True) # Converte números em texto
    citations = [i for i, _ in hits] #Extrai indices dos documentos
    return answer, citations

# Teste ponta-a-ponta (RAG -> geração)
question = "O que é mitose e quais são suas fases?" #Prompt de entrada do usuário
hits = retrieve(question, k=3) #Busca no Faiss a Pergunta convertida em embedding , traz top 3 de docs relevantes
ans, cits = generate_primary(question, hits) #Pega todo o contexto e utiliza t5 para gerar a resposta obedecendo os prompts e regras de saida
print("Resposta (primária):", ans) # Printa resposta
print("Citações (docs usados):", cits) # Printa os Docs de onde tirou contexto


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Resposta (primária): a mudança gradual na comunidade ao longo do tempo, podendo ser primária ou secundária.
Citações (docs usados): [2, 6, 4]


---
## 🛡️ Aula 3 — Validação (Pydantic + heurísticas) + Aderência semântica

**Objetivo**: Rejeitar respostas ruins/fora do contexto e padronizar a saída.

**Conceitos & técnicas**:
- Pydantic para esquema de resposta
- Heurísticas: mínimo de palavras, frases proibidas, exige citações
- Aderência: similaridade (embedding) entre resposta e contexto


In [ ]:
# Célula 7 — Instalação para validação
!pip install -q pydantic


In [ ]:
# Célula 8 — Pydantic + heurísticas + aderência
from pydantic import BaseModel, Field, ValidationError
from typing import List, Literal

class AnswerModel(BaseModel): #Base Model : Classe base para criar modelos de dados , Pydantic.
    answer: str = Field(..., min_length=20) # Define um campo string , Field Obrigatório numero minimo de caracteres
    citations: List[int] = Field(default_factory=list) # Cria uma Lista de inteiros , o padrão é uma lista vazia
    confidence: float = Field(..., ge=0, le=1) ##
    mode: Literal["primary","fallback"] = "primary" ##

BANNED = [
    "não tenho informações", "sou apenas uma ia", "não posso ajudar",
    "como ia", "desculpe, não sei"
] # Lista de respostas ruins , se detectadas reprovam a resposta

def text_similarity(a: str, b: str) -> float:
    # Similaridade de cosseno entre embeddings normalizados
    va = embedder.encode([a], convert_to_numpy=True, normalize_embeddings=True)[0] # Converte texto A em vetor (384D) ,  Formato NumPy array , Normaliza vetor = 1
    vb = embedder.encode([b], convert_to_numpy=True, normalize_embeddings=True)[0]
    return float(np.dot(va, vb))  # [-1,1] ~ aqui ~[0,1] na prática

def validate_answer(answer: str, citations: List[int], context_text: str,
                    min_words=10, min_sim=0.55): # Answer : Resposta gerada pelo t5 , Citations : Indices dos docs , contexto concatenado , minimo de palavras e minimo de similaridade set 0.55
    reasons = [] # Lista vazia , aqui vai os motivos da reprovação.

    # 1) tamanho mínimo
    if len(answer.split()) < min_words:
        reasons.append("resposta muito curta") # Splita o texto e valida se a respsota é muito curta.

    # 2) frases proibidas
    low = answer.lower()
    if any(p in low for p in BANNED): # detecta as frases proibidas (OBS : Transformar em lower case para evitar case sensitive)
        reasons.append("frase proibida detectada")

    # 3) citações obrigatórias
    if not citations:
        reasons.append("sem citações de contexto") # Verifica se tem citacoes dos nossos docs

    # 4) aderência semântica (resposta ↔ contexto)
    sim = text_similarity(answer, context_text)
    if sim < min_sim:
        reasons.append(f"aderência baixa (sim={sim:.2f} < {min_sim})")

    ok = len(reasons) == 0
    # confiança simples: clip do score de similaridade , setada em 0.55 nota corte
    confidence = max(0.0, min(1.0, (sim - 0.4) / 0.6))  # mapeia ~[0.4..1.0] → [0..1]
    return ok, reasons, confidence, sim

# Teste de validação
q = "Explique biodiversidade e sua relação com resiliência ecológica."
hits = retrieve(q, k=3) # Busca top 3 Docs FAISS
ans, cits = generate_primary(q, hits) # Gera Resposta com T5
ctx = build_context(hits) #Concatena Docs recuperados

ok, reasons, conf, sim = validate_answer(ans, cits, ctx)
print("OK?", ok, "| conf:", round(conf,2), "| sim:", round(sim,2), "| motivos:", reasons)

if ok:
    try:
        obj = AnswerModel(answer=ans, citations=cits, confidence=conf, mode="primary") #Try de criação objeto Pydantic
        print("✅ Struct Pydantic:", obj.model_dump())
    except ValidationError as e:
        print("❌ Pydantic:", e)
else:
    print("❌ Reprovado na validação:", reasons)


OK? False | conf: 0.03 | sim: 0.42 | motivos: ['aderência baixa (sim=0.42 < 0.55)']
❌ Reprovado na validação: ['aderência baixa (sim=0.42 < 0.55)']


---
## 🔄 Aula 4 — Fallback Router + Interface (Gradio)

**Objetivo**: Se a validação reprovar, tentar fallback com outro modelo e exibir numa UI simples.

**Conceitos & técnicas**:
- Fallback: tentar `google/flan-t5-base` com prompt alternativo
- Router: `primary → validate → (se fail) fallback → validate`
- Interface: Gradio, mostrando resposta, modo e documentos citados


In [4]:
# Célula 9 — Instalação de UI e segundo modelo
!pip install -q gradio transformers


In [5]:
# Célula 10 — Modelo fallback (FLAN-T5) e geração alternativa
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

FALLBACK_MODEL_ID = "google/flan-t5-base"
tok_fallback = AutoTokenizer.from_pretrained(FALLBACK_MODEL_ID)
mdl_fallback = AutoModelForSeq2SeqLM.from_pretrained(FALLBACK_MODEL_ID)

def generate_fallback(question: str, hits, max_in=512, max_out=180):
    context = build_context(hits)
    prompt = (
        "Responda em português, de forma objetiva e didática, usando exclusivamente o CONTEXTO. "
        "Se faltar informação, diga: 'Não encontrei no contexto.' "
        f"Pergunta: {question} "
        f"Contexto: {context}"
    )
    inputs = tok_fallback(prompt, return_tensors='pt', truncation=True, max_length=max_in)
    outputs = mdl_fallback.generate(**inputs, max_length=max_out, num_beams=6, early_stopping=True)
    answer = tok_fallback.decode(outputs[0], skip_special_tokens=True)
    citations = [i for i, _ in hits]
    return answer, citations


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [6]:
# Célula 11 — Router (primary -> validate -> fallback se necessário)
def route_and_answer(question: str, k: int = 3):
    hits = retrieve(question, k=k)
    context = build_context(hits)

    # 1) primary
    a1, c1 = generate_primary(question, hits)
    ok, reasons, conf, sim = validate_answer(a1, c1, context)
    if ok:
        obj = AnswerModel(answer=a1, citations=c1, confidence=conf, mode="primary")
        return obj, hits, reasons

    # 2) fallback
    a2, c2 = generate_fallback(question, hits)
    ok2, reasons2, conf2, sim2 = validate_answer(a2, c2, context)
    mode = "fallback"

    if ok2:
        obj = AnswerModel(answer=a2, citations=c2, confidence=conf2, mode=mode)
        return obj, hits, reasons2
    else:
        # Ajuste simples para cumprir min_length quando reprova
        a2_adj = (a2 + " (Reveja a pergunta ou adicione mais contexto.)") if len(a2) < 20 else a2
        conf2 = max(conf2, 0.01)
        obj = AnswerModel(answer=a2_adj, citations=c2, confidence=conf2, mode=mode)
        return obj, hits, reasons2


In [7]:
# Célula 12 — UI com Gradio
import gradio as gr

def ui_ask(question):
    obj, hits, reasons = route_and_answer(question, k=3)
    used = "\n".join([f"- [{i}] {documents[i]}" for i, _ in hits])
    extra = "" if not reasons else ("\n\n⚠️ Observações da validação: " + "; ".join(reasons))
    out = (
        f"**Modo:** {obj.mode}\n"
        f"**Confiança:** {obj.confidence:.2f}\n"
        f"**Citações:** {obj.citations}\n\n"
        f"**Resposta:**\n{obj.answer}\n\n"
        f"**Trechos usados:**\n{used}"
        f"{extra}"
    )
    return out

demo = gr.Interface(
    fn=ui_ask,
    inputs=gr.Textbox(label="Pergunte sobre Biologia", placeholder="Ex.: O que é fotossíntese?"),
    outputs=gr.Markdown(label="Resposta"),
    title="Chatbot de Biologia — RAG + Guardrails + Fallback"
)
demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0c41697c2e15aaff14.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


---
### ✅ O que você terá ao final
- **RAG simples** com documentos de **biologia**, FAISS e top-k.
- **LLM primária (T5)** respondendo **apenas com o contexto**.
- **Validação (Pydantic + heurísticas)** garantindo qualidade e aderência.
- **Fallback router** (FLAN-T5) quando a resposta reprovar.
- **Interface Gradio** para o usuário interagir e ver **modo + citações**.

Dica: no Colab, se quiser **expor um endpoint** temporário, pode usar `gradio` com `share=True` (como no exemplo) ou soluções como `ngrok`.
